<a href="https://colab.research.google.com/github/AntoScher/L-PE6.2-LangChain-google/blob/main/%D0%A0%D0%956_3forGem1_5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install google-auth==2.38.0
!pip install -qU langchain-google-genai>=0.1.3 chromadb>=0.4.16 tiktoken>=0.7.0
!pip install langchain>=0.2.0 langchain-core>=0.2.0 langchain-text-splitters langchain-community
!pip show chromadb langchain langchain-community

Name: chromadb
Version: 1.0.12
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: bcrypt, build, fastapi, grpcio, httpx, importlib-resources, jsonschema, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, pyyaml, rich, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 
---
Name: langchain
Version: 0.3.25
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
---
Name: langchain-community
Version: 0.3.

In [1]:
import os
import chromadb
from google.colab import userdata

# LangChain компоненты
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma  # Импорт из основного модуля LangChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Инициализация Chroma
client = chromadb.Client()

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


In [3]:
# Исходный текст
text = '''Благодаря появлению LLM возникла новая профессия будущего.
Это промпт-инженер, который наилучшим образом направляет LLM на правильный путь.
Этот специалист должен обладать творческим мышлением и аналитическими способностями,
и желательно знанием методов машинного обучения и NLP и навыками программирования'''

# Разделение на чанки
splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=100,
    separators=['.\n']
)
splits = splitter.split_text(text)

# Инициализация моделей
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.7)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Создание векторного хранилища
vectorstore = Chroma.from_texts(
    texts=splits,
    embedding=embeddings,
    collection_name="my_rag_collection",
    client=client,
    persist_directory=None
)

# Проверка
test_embedding = embeddings.embed_query("test")
print(f"Размерность эмбеддингов: {len(test_embedding)}")  # Должно быть 768

Размерность эмбеддингов: 768


In [4]:
def rag_query(question: str, k=2):
    docs = vectorstore.similarity_search(question, k=k)  # Используем vectorstore
    return "\n\n".join([f"Документ {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])

prompt_template = """
Ты - эксперт по анализу текстов. Ответь на вопрос, используя только предоставленные документы:

{context}

Вопрос: {question}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
chain = (
    {"context": rag_query, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [5]:
print("Ответ 1:")
print(chain.invoke("Кто такой промпт-инженер?"))

print("\nОтвет 2:")
print(chain.invoke("Какие навыки нужны промпт-инженеру?"))

Ответ 1:
Промпт-инженер — это специалист, который наилучшим образом направляет LLM на правильный путь.

Ответ 2:
Творческое мышление, аналитические способности, желательно знание методов машинного обучения и NLP, и навыки программирования.
